# ScotRail Analysis with Ibis

This notebook uses [ibis-datasette](https://github.com/jcrist/ibis-datasette) to analyze data from the [ScotRail](https://scotrail.datasette.io/) [datasette](datasette.io/).

This datasette is _super_ fun to play around with. It's composed of ~2400 different audioclips (and transcriptions) from Scottish train operator ScotRail's automated station announcements.

If you haven't seen it, I encourage you to read Simon Willison's [excellent blogpost](https://simonwillison.net/2022/Aug/21/scotrail/) on putting this datasette together, and some interesting queries to try (we'll be replicating one of these below).

While you can use the [datasette UI](https://scotrail.datasette.io/) directly, I wanted to use [ibis](https://ibis-project.org) and the full power of Python to explore and build some interesting things.

---

Like most notebooks, first we start with some imports and initialization.

Here we:

- Import `ibis` and its `_` helper (more on this later)
- Enable ibis's interactive mode
- We also tweak pandas' display options to better render wide columns. This makes the transcriptions below easier to read.

In [ ]:
import ibis
from ibis import _

ibis.options.interactive = True

import pandas as pd
pd.set_option('max_colwidth', 400)

Next we need to connect to the datasette. This is done by passing the full URL to `ibis.datasette.connect`:

In [ ]:
con = ibis.datasette.connect("https://scotrail.datasette.io/scotrail")

Once connected, we can start poking around.

The first thing I usually do when exploring a new datasette is examine the tables and schemas:

In [ ]:
con.list_tables()

In [ ]:
con.tables.announcements.schema()

In [ ]:
 con.tables.announcements.head()

The main table is `announcments`, the most interesting columns of which are:

- `Transcription`: a full transcription of the audio clip
- `Category`: a category that the audio clip belongs to
- `mp3`: a link to the audio clip, hosted on GitHub

Since we're going to be accessing this table a lot below, lets save it to a shorter local variable name:

In [ ]:
t = con.tables.announcements

To get a better sense of the scale of data we're working with, lets take a closer look at the `Category` column.

I want to know how many categories there are, and how the audio clips are distributed across these categories.

To do this, we can use:

- `.group_by("Category")` to split the data into separate groups by `Category`
- `.count()` to then count how many rows are in each category.
- `.sort_by(ibis.desc("count"))` to then sort the rows by `count`, descending.

In [ ]:
category_counts = (
    t.group_by("Category")
     .count()
     .sort_by(ibis.desc("count"))
)

category_counts

Here we can see there are 23 categories, with 90% of the audio clips falling into the first 6. A few categories to highlight:

- `Destination` is a ScotRail stop
- `Reason` is a reason for a cancellation. These are fun to look through.
- `Passenger information` is a bit of miscellaneous. ("The train is ready to leave" for example)
- `Number` and `Time` are just clips of saying numbers and times
- `Train operating company` is the name of a train operating company
- `Apology` is the start of an apology for a service disruption ("I am sorry to announce that the" for example)

The `Reason` category is the most fun to look through. There are all sorts of reasons a train might be cancelled, from "Sheep on the railway" to "A wartime bomb near the railway".

---

One reoccuring reason is theft (err, "attempted theft") of various things. Lets find all reasons involving "theft". 

This can be done by using `.filter()` to filter rows based on a predicate. Here we need two predicates:

- `_.Category == "Reason"` selects all rows that have a category of "Reason"
- `_.Transcription.contains("theft")` selects all rows with a transcription containing the string "theft"

In [ ]:
thefts = t.filter((_.Category == "Reason") & _.Transcription.contains("theft"))

thefts

All of these rows also include a link to an `mp3` file containing that clip. To play a clip in a jupyter notebook, we can make use of `IPython.display.Audio`. For example, lets play the first clip from above:

In [ ]:
from IPython.display import Audio

mp3_url = thefts.limit(1).execute().mp3.iloc[0]

Audio(mp3_url)

## Generating a Random Apology

In [his blogpost](https://simonwillison.net/2022/Aug/21/scotrail/) Simon wrote up a SQL query for generating a Random apology by combining a few random rows from different categories above. It generates surprisingly coherent sentences, you can see the datasette version [here](https://scotrail.datasette.io/scotrail/random_apology).

If you're interested you can click `show` at the top to see the full SQL query - it's readable, but a bit long.

I wanted to reproduce the same query using `ibis`. Since `ibis` is just a Python library, you can make use of things like functions to abstract away some of the repetitiveness in the SQL query above.

Here's what I came up with:

In [ ]:
def random(category):
    """Select a random row from a given category"""
    return (
        t.filter(_.Category == category)
         .sort_by(ibis.random())
         .select("Transcription", "mp3")
         .limit(1)
    )

def phrase(text):
    """Select a row with a specific transcription"""
    return (
        t.filter(_.Transcription == text)
         .select("Transcription", "mp3")
         .limit(1)
    )

query = ibis.union(
    random("Apology"),
    random("Train operating company"),
    random("Destination"),
    phrase("has been cancelled"),
    phrase("due to"),
    random("Reason"),
)

Since the query selects random rows, if you run the cell below multiple times, you should see different results every time:

In [ ]:
query.execute()

If we wanted to do all computation in the backend, we could use `group_concat` ([docs](https://www.sqlite.org/lang_aggfunc.html#group_concat)) to then concatenate the Transcription rows together, returning a single string:

In [ ]:
random_apology = query.Transcription.group_concat(" ")

random_apology

Note that the full query above is translated to SQL and executed on the `datasette` server, no computation is happening locally.

If you want to see the generated SQL, you can use the `ibis.show_sql` function:

In [ ]:
ibis.show_sql(random_apology)

However, we're only using `ibis` to push the bulk of the computation to the backend. We don't need to handle _everything_ in SQL, only enough to reduce the size of the results to something reasonable to return from the `datasette` server.

We also have access to the full Python ecosystem to process results. This lets us do some things that wouldn't be possible in SQL alone, like concatenating `mp3` files :).

## A "Random Apology" Button

The [ipywidgets](https://ipywidgets.readthedocs.io) library provides support for building simple UIs in Python, with the rendering handled by the notebook. This is nice for me, as I am _not_ a web engineer - I'm a novice at best at javascript/html. However, I do know how to write Python.

Below we hack together a quick UI with `ipywidgets` to make a button for generating a random apology, complete with a merged `mp3` file so you can listen to your work. You don't really need to understand this code, it has nothing to do with `ibis` or `ibis-datasette` itself.

Clicking the button will pull generate a new random apology, download and merge the mp3 files, and display both the apology sentence and merged mp3.

In [ ]:
import tempfile
import os
import pydub
import httpx
import ipywidgets
from IPython.display import Audio, display

output = ipywidgets.Output()
button = ipywidgets.Button(description='Random Apology', icon="repeat")
UI = ipywidgets.VBox([button, output])


def concatenate_mp3s(urls: list[str]) -> bytes:
    with httpx.Client(follow_redirects=True) as client, tempfile.TemporaryDirectory() as tempdir:
        output = None
        for i, url in enumerate(urls):
            path = os.path.join(tempdir, f"part{i}.mp3")
            with open(path, "wb") as f:
                resp = client.get(url)
                resp.raise_for_status()
                f.write(resp.content)
            part = pydub.AudioSegment.from_mp3(path)
            if output is None:
                output = part
            else:
                output = output + part
        out_path = os.path.join(tempdir, "output.mp3")
        output.export(out_path, format="mp3")
        with open(out_path, "rb") as f:
            return f.read()


@button.on_click
def on_click(*args):
    output.clear_output()
    result = query.execute()
    msg = " ".join(result.Transcription)
    mp3 = concatenate_mp3s(result.mp3)
    with output:
        print(msg)
        display(Audio(mp3))

        
UI

## Review

`datasette` makes it easier to publish accessible open data on the web, with a UI exposed for writing SQL queries. However, not everyone is extremely SQL literate (myself included). `ibis` and `ibis-datasette` let Python programmers access this same data resource, but through a familiar dataframe-like interface.

For more information on `ibis`, see the [official documentation](https://ibis-project.org).